In [1]:
periodo = 'pre-pandemia'

# Importe as bibliotecas necessárias

In [2]:
%matplotlib inline
from IPython.display import clear_output
import tqdm
import os
import pandas as pd
import numpy as np
import seaborn as sns
import random
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer 
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree._classes import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neighbors._classification import KNeighborsClassifier
from sklearn.neural_network._multilayer_perceptron import MLPClassifier
from sklearn.linear_model._logistic import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from yellowbrick.style.colors import resolve_colors
from sklearn.model_selection._validation import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from unidecode import unidecode
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.decomposition import PCA
import pickle
#import statsmodels.api as sm
import platform 
np.random.seed(42)

In [3]:
def strfdelta(tdelta, fmt="{hours}:{minutes}:{seconds}"):
    d = {"days": tdelta.days}
    d["hours"], rem = divmod(tdelta.seconds, 3600)
    d["minutes"], d["seconds"] = divmod(rem, 60)
    return fmt.format(**d)

# Carregando Base de dados

In [4]:
df = pd.read_csv(f'twitterbase_{periodo}.csv', sep=';')
df.head()

,volumeTweets_media,volumeTweets_variancia,volumeTweets_mediaMovelPonterada,volumeTweets_entropia,indiceInsonia_media,indiceInsonia_variancia,indiceInsonia_mediaMovelPonterada,indiceInsonia_entropia,pronome1Pessoa_media,pronome1Pessoa_variancia,...,links_entropia,midia_media,midia_variancia,midia_mediaMovelPonterada,midia_entropia,curtidas_media,curtidas_variancia,curtidas_mediaMovelPonterada,curtidas_entropia,classe
0,0.232915,0.315346,1.605634,0.593198,0.086239,0.111151,0.578638,0.307958,0.016736,0.022035,...,0.324054,0.006974,0.006925,0.049296,0.041577,0.103208,0.179027,0.725352,0.320571,depressao
1,2.510294,47.111659,17.270431,1.838695,0.620732,1.595054,4.288012,1.377158,0.483824,3.058562,...,0.159815,0.000000,0.000000,0.000000,0.000000,1.048529,5.416763,7.237741,1.303106,depressao
2,0.356322,1.985678,2.425234,0.612006,0.146022,0.323983,1.003827,0.443252,0.112644,0.251679,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.002299,0.002294,0.002336,0.016263,controle
3,0.154891,0.891770,1.027701,0.334143,0.034081,0.039944,0.243190,0.160835,0.008152,0.013520,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,1.033288,0.891967,0.183438,controle
4,30.116279,1931.916712,206.861111,2.842062,2.232019,15.639582,16.762029,2.242101,10.953488,269.858302,...,2.348284,0.441860,1.176852,3.388889,0.795581,4.023256,75.557599,25.500000,1.857342,depressao


# Codificando a variável categórica

In [5]:
label_encoder = LabelEncoder()
df['classe'] = label_encoder.fit_transform(df['classe']).astype('float64')

# Preparando os dados

In [6]:
def train_val_test(X, y, train_ratio, val_ratio, test_ratio):
    assert sum([train_ratio, test_ratio, val_ratio])==1.0, "wrong given ratio, all ratios have to sum to 1.0"
    assert X.shape[0]==len(y), "X and y shape mismatch"

    ind_train = int(round(X.shape[0]*train_ratio))
    ind_test = int(round(X.shape[0]*(train_ratio+test_ratio)))

    X_train = X[:ind_train]
    X_test = X[ind_train:ind_test]
    X_val = X[ind_test:]

    y_train = y[:ind_train]
    y_test = y[ind_train:ind_test]
    y_val = y[ind_test:]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [7]:

def avaliacao(classificador, predicao, x, y, nome_classificador):
    '''
    Computa as mericas de avaliação dos clasficadores
    '''
        
    precisao = precision_score(y, predicao)
    revocacao = recall_score(y, predicao)
    f1score = f1_score(y, predicao)
    acuracia = accuracy_score(y, predicao)
    return [precisao, revocacao, f1score, acuracia]


    
def modeloML(model, x, x_train, y_train, x_val, y_val, nome_classificador, atributo, i):
    clear_output()
    
    print(f'{atributo}\nExperimento {i}')
    print(nome_classificador)
    
    model.fit(x_train, y_train)
    predicao = cross_val_predict(model, x_val, y_val, cv=cv)
    #predicao = GridSearchCV(model, {}, cv=cv, verbose=10, n_jobs=3).fit(x_train, y_train).predict(x_val)
    #predicao = cross_val_predict(GridSearchCV(model, {}, cv=cv, verbose=10).fit(x_train, y_train).best_estimator_, x_val, y_val, cv=cv)
    resultados = avaliacao(model, predicao, x_val, y_val, nome_classificador)
    df_resultados.loc[nome_classificador][f'DeChoudhury_+_{atributo}_precisao'] = round(resultados[0] * 100, 1)
    df_resultados.loc[nome_classificador][f'DeChoudhury_+_{atributo}_recall'] = round(resultados[1] * 100, 1)
    df_resultados.loc[nome_classificador][f'DeChoudhury_+_{atributo}_f1_score'] = round(resultados[2] * 100, 1)
    df_resultados.loc[nome_classificador][f'DeChoudhury_+_{atributo}_acuracia'] = round(resultados[3] * 100, 1)
    
    #plotarMatrizConfusao(y_val, predicao, nome_classificador)

In [8]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)
zscore = StandardScaler()

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

In [9]:
if not os.path.exists(f'df_resultados_{periodo}_novosAtributos_f1Score.xlsx'):
    df_choudhury = pd.read_excel(f'df_resultados_{periodo}_f1Score.xlsx', index_col=0)
    df_resultados = df_choudhury.loc[['Gradient Boosting', 'Perceptron Multicamadas']]
    atributos = ['caracteresOrientais', 'emojis', 'links','midia', 'curtidas']
    metricas = ['acuracia', 'precisao', 'recall', 'f1_score']
    colunas = [f'DeChoudhury_+_{i}_{j}' for i in atributos for j in metricas]
    df_resultados_novosAtributos = pd.DataFrame(columns=colunas, index=['Gradient Boosting', 'Perceptron Multicamadas'], data=np.nan)
    df_resultados = pd.concat([df_resultados, df_resultados_novosAtributos], axis=1, join='inner')
    colunas = [f'DeChoudhury_+_todosAtributos_{i}'for i in metricas]
    df_resultados_novosAtributos = pd.DataFrame(columns=colunas, index=['Gradient Boosting', 'Perceptron Multicamadas'], data=np.nan)
    df_resultados = pd.concat([df_resultados, df_resultados_novosAtributos], axis=1, join='inner')
else:
    df_resultados = pd.read_excel(f'df_resultados_{periodo}_novosAtributos_f1Score.xlsx', index_col=0)

df_resultados

,exp_basico_DeChoudhury_acuracia,exp_basico_DeChoudhury_precisao,exp_basico_DeChoudhury_recall,exp_basico_DeChoudhury_f1_score,DeChoudhury_+_caracteresOrientais_acuracia,DeChoudhury_+_caracteresOrientais_precisao,DeChoudhury_+_caracteresOrientais_recall,DeChoudhury_+_caracteresOrientais_f1_score,DeChoudhury_+_emojis_acuracia,DeChoudhury_+_emojis_precisao,...,DeChoudhury_+_midia_recall,DeChoudhury_+_midia_f1_score,DeChoudhury_+_curtidas_acuracia,DeChoudhury_+_curtidas_precisao,DeChoudhury_+_curtidas_recall,DeChoudhury_+_curtidas_f1_score,DeChoudhury_+_todosAtributos_acuracia,DeChoudhury_+_todosAtributos_precisao,DeChoudhury_+_todosAtributos_recall,DeChoudhury_+_todosAtributos_f1_score
Gradient Boosting,77.2,75.5,90.1,82.2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Perceptron Multicamadas,76.1,75.5,87.4,81.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def executar(x, y, atributo, i):
    
    x_train, y_train, x_val, y_val, x_test, y_test = train_val_test(x, y, train_ratio, validation_ratio, test_ratio)
    
    naiveBayes = BernoulliNB(alpha=1e-09)
    gradientBoosting = GradientBoostingClassifier(random_state=42, n_estimators=100, verbose=10)
    perceptronMulticamadas = MLPClassifier(random_state=42, hidden_layer_sizes=(10,30,10), max_iter=100, verbose=10)
    
    modeloML(perceptronMulticamadas, x, x_train, y_train, x_val, y_val, 'Perceptron Multicamadas', atributo, i)
    modeloML(gradientBoosting, x, x_train, y_train, x_val, y_val, 'Gradient Boosting', atributo, i)

In [11]:
choq = ['volumeTweets_media', 'volumeTweets_mediaMovelPonterada', 
        'indiceInsonia_variancia', 'indiceInsonia_mediaMovelPonterada',
        'pronome1Pessoa_media', 'pronome1Pessoa_variancia', 
        'pronome2Pessoa_mediaMovelPonterada', 'pronome2Pessoa_entropia',
        'pronome3Pessoa_media', 'pronome3Pessoa_mediaMovelPonterada',
        'valencia_mediaMovelPonterada', 'valencia_entropia', 
        'ativacao_mediaMovelPonterada', 'ativacao_entropia', 
        'termosDepressivos_variancia', 'termosDepressivos_mediaMovelPonterada',
        'grafoSocial_variancia', 'grafoSocial_mediaMovelPonterada',
        'medicamentosAntiDepressivo_media', 'medicamentosAntiDepressivo_mediaMovelPonterada']
caracteresOrientais = ['caracteresOrientais_variancia', 'caracteresOrientais_mediaMovelPonterada']
emojis = ['emojis_variancia', 'emojis_entropia']
links = ['links_mediaMovelPonterada', 'links_entropia']
midia = ['midia_variancia', 'midia_mediaMovelPonterada']
curtidas = ['curtidas_media', 'curtidas_mediaMovelPonterada']

In [12]:
exp1 = choq + caracteresOrientais
exp2 = choq + emojis
exp3 = choq + links
exp4 = choq + midia
exp5 = choq + curtidas
exp6 = choq + caracteresOrientais + emojis + links + midia + curtidas

exps = [exp1, exp2, exp3, exp4, exp5, exp6]
atributos = ['caracteresOrientais', 'emojis', 'links', 'midia', 'curtidas', 'todosAtributos']

lista = list(zip(exps, atributos))

for i, (exp, atributo) in enumerate(lista, 1):
    x = df[exp].copy()
    y = df['classe'].copy()
    x_new = zscore.fit_transform(x)
    executar(x_new, y, atributo, i)

df_resultados_round = df_resultados.copy()
df_resultados_round.to_excel(f'df_resultados_{periodo}_novosAtributos_f1Score.xlsx', encoding='utf-8')

todosAtributos
Experimento 6
Gradient Boosting
      Iter       Train Loss   Remaining Time 
         1           1.3297           44.59s
         2           1.2842           43.87s
         3           1.2465           43.95s
         4           1.2145           43.20s
         5           1.1877           42.74s
         6           1.1647           42.29s
         7           1.1446           41.87s
         8           1.1279           41.26s
         9           1.1125           40.83s
        10           1.0986           40.43s
        11           1.0873           39.99s
        12           1.0768           39.43s
        13           1.0682           39.00s
        14           1.0600           38.57s
        15           1.0524           38.14s
        16           1.0457           37.87s
        17           1.0398           37.51s
        18           1.0347           37.20s
        19           1.0301           36.82s
        20           1.0254           36.42s
       

        81           0.9046            1.49s
        82           0.9032            1.41s
        83           0.9020            1.33s
        84           0.9009            1.26s
        85           0.8996            1.18s
        86           0.8984            1.10s
        87           0.8966            1.02s
        88           0.8962            0.94s
        89           0.8952            0.86s
        90           0.8941            0.79s
        91           0.8928            0.71s
        92           0.8919            0.63s
        93           0.8901            0.55s
        94           0.8890            0.47s
        95           0.8880            0.39s
        96           0.8877            0.31s
        97           0.8873            0.24s
        98           0.8866            0.16s
        99           0.8856            0.08s
       100           0.8835            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3052            7.73s
         

        64           0.9160            2.89s
        65           0.9147            2.81s
        66           0.9118            2.73s
        67           0.9103            2.65s
        68           0.9093            2.57s
        69           0.9082            2.49s
        70           0.9066            2.41s
        71           0.9056            2.32s
        72           0.9052            2.24s
        73           0.9048            2.17s
        74           0.9035            2.09s
        75           0.9023            2.01s
        76           0.9011            1.93s
        77           0.8997            1.85s
        78           0.8983            1.76s
        79           0.8964            1.69s
        80           0.8941            1.60s
        81           0.8931            1.52s
        82           0.8923            1.45s
        83           0.8912            1.36s
        84           0.8906            1.28s
        85           0.8888            1.20s
        86

        47           0.9461            4.60s
        48           0.9445            4.51s
        49           0.9431            4.42s
        50           0.9420            4.34s
        51           0.9410            4.24s
        52           0.9393            4.15s
        53           0.9381            4.07s
        54           0.9354            3.98s
        55           0.9334            3.89s
        56           0.9328            3.81s
        57           0.9316            3.71s
        58           0.9301            3.62s
        59           0.9288            3.54s
        60           0.9277            3.46s
        61           0.9259            3.36s
        62           0.9246            3.28s
        63           0.9227            3.19s
        64           0.9214            3.10s
        65           0.9209            3.02s
        66           0.9197            2.94s
        67           0.9183            2.85s
        68           0.9170            2.77s
        69

        30           0.9724            6.09s
        31           0.9701            6.02s
        32           0.9677            5.91s
        33           0.9655            5.84s
        34           0.9628            5.74s
        35           0.9606            5.66s
        36           0.9574            5.56s
        37           0.9552            5.46s
        38           0.9535            5.36s
        39           0.9517            5.28s
        40           0.9491            5.19s
        41           0.9473            5.09s
        42           0.9457            5.01s
        43           0.9442            4.93s
        44           0.9427            4.84s
        45           0.9405            4.74s
        46           0.9384            4.67s
        47           0.9366            4.57s
        48           0.9357            4.49s
        49           0.9341            4.40s
        50           0.9328            4.32s
        51           0.9310            4.23s
        52

        13           1.0547            7.27s
        14           1.0461            7.13s
        15           1.0391            7.02s
        16           1.0327            7.01s
        17           1.0266            6.89s
        18           1.0204            6.78s
        19           1.0149            6.77s
        20           1.0108            6.64s
        21           1.0068            6.60s
        22           1.0024            6.52s
        23           0.9984            6.40s
        24           0.9939            6.35s
        25           0.9902            6.27s
        26           0.9872            6.15s
        27           0.9836            6.10s
        28           0.9807            6.03s
        29           0.9777            5.95s
        30           0.9752            5.85s
        31           0.9726            5.78s
        32           0.9700            5.66s
        33           0.9678            5.60s
        34           0.9653            5.53s
        35

        96           0.8856            0.34s
        97           0.8852            0.26s
        98           0.8843            0.17s
        99           0.8832            0.09s
       100           0.8825            0.00s


In [13]:
colunasFf1Score = [col for col in df_resultados.columns if 'f1_score' in col]
df_resultados_round[colunasFf1Score].style.set_precision(1)

,exp_basico_DeChoudhury_f1_score,DeChoudhury_+_caracteresOrientais_f1_score,DeChoudhury_+_emojis_f1_score,DeChoudhury_+_links_f1_score,DeChoudhury_+_midia_f1_score,DeChoudhury_+_curtidas_f1_score,DeChoudhury_+_todosAtributos_f1_score
Gradient Boosting,82.2,82.2,82.1,82.0,82.2,82.1,82.3
Perceptron Multicamadas,81.0,81.1,81.0,81.2,81.0,81.1,80.9
